In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")

import os
import sys; sys.path.append('../')
import numpy as np
from numpy.random import default_rng
from numpy.fft import fft, ifft

import scipy.io as sio
import pysindy as ps
from PDE_FIND import build_linear_system, print_pde, TrainSTRidge
from robust_PDE_FIND import RobustPCA
from best_subset import *
from UBIC import *

from sklearn.decomposition import PCA, TruncatedSVD
from skimage.restoration import denoise_wavelet, estimate_sigma

def evaluate_coefficients(prediected_coeffs):
    GROUND = np.array([0.1, -1])
    errs = 100*np.abs(GROUND-np.array(prediected_coeffs).flatten())/np.abs(GROUND)
    return errs.mean(), errs.std()

Sklearn's version: 1.2.2


#### Load data

In [2]:
data = sio.loadmat('../Datasets/burgers.mat')
u = (data['usol']).real
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]
dx = x[2]-x[1]
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

#### Visualization

In [3]:
with plt.style.context(['science']):
    ufig, uax = plt.subplots()
    contour = uax.pcolormesh(t, x, u, shading='gouraud')
    uax.set_xlabel("$t$")
    uax.set_ylabel("$x$")
    uax.set_title("$u$")
    cbar = ufig.colorbar(contour)
    # ufig.savefig("../Figures/burgers.png", dpi=300)
    plt.show()

#### Add noise

In [4]:
print("----- Clean -----")
library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2,
    spatial_grid=x,
    is_uniform=True,
)

print('STLSQ model: ')
optimizer = ps.STLSQ(threshold=0.1, alpha=1e-5, normalize_columns=False)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(np.expand_dims(u, -1), t=dt)
model.print()

Ut, R, rhs_des = build_linear_system(u, dt, dx, D=3, P=3, time_diff = 'FD', space_diff = 'FD')
rhs_des = rhs_des[1:]
R = R[:, 1:].real; Ut = Ut.real
w = TrainSTRidge(R,Ut,10**-5,0.1)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

print()

print("----- Add 30% noise -----")
np.random.seed(0)
noise_lv = float(30)
print(noise_lv)
noise = 0.01*np.abs(noise_lv)*(u.std())*np.random.randn(u.shape[0],u.shape[1])
un = u + noise
np.mean(noise), np.std(noise), np.max(noise), np.min(noise)

Utn, Rn, rhs_des = build_linear_system(un, dt, dx, D=3, P=3, time_diff = 'poly',
                                       deg_x = 4, deg_t = 4, 
                                       width_x = 10, width_t = 10)
rhs_des = rhs_des[1:]
Rn = Rn[:, 1:].real; Utn = Utn.real
# Solve with STRidge
w = TrainSTRidge(Rn,Utn,10**-5,1)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

----- Clean -----
STLSQ model: 
(x0)' = 0.100 x0_11 + -1.001 x0x0_1
l0_penalty: 1.6123253173828125
PDE derived using STRidge
u_t = (-1.000403 +0.000000i)uu_{x}
    + (0.100145 +0.000000i)u_{xx}
   

----- Add 30% noise -----
30.0
l0_penalty: 1.93969921875
PDE derived using STRidge
u_t = (-0.775461 +0.000000i)uu_{x}
   


In [5]:
print(np.linalg.norm(u-un, ord='fro'), np.linalg.norm(u-un, ord=np.inf))

8.673577639876537 5.298318588581592


#### Use the precomputed denoised variable given by the K-SVD

In [6]:
denoised = True
if denoised: 
#     denoised_un = np.load("./denoised_files/burgers_dictlearn_denoised30_components32_8x8.npy")
#     un = denoised_un
    
    ### PCA ###
#     target_percentage = 99.5
#     Sigma = PCA(n_components=un.shape[-1]).fit(un).explained_variance_ratio_
#     # sigma = Sigma/Sigma.sum() # if .explained_variance_ is used above
#     summ = np.cumsum(100*Sigma)
#     n_components = np.argmax(summ>target_percentage)+1
#     assert n_components < len(Sigma)
#     print(n_components, summ[n_components-1])
#     pca = PCA(n_components=n_components).fit(un)
#     pca_un = pca.inverse_transform(pca.transform(un))
#     un = pca_un
    
    ### SVD ###
#     target_percentage = 99.99 # 90, 95, 99.99
#     U, Sigma, V = np.linalg.svd(un, full_matrices=False); V = V.T
#     sigma = (Sigma.copy())**2
#     sigma = sigma/sigma.sum()
#     summ = np.cumsum(100*sigma)
#     n_components = np.argmax(summ>target_percentage)+1
#     assert n_components < len(Sigma)
#     svd_un = U[:, 0:n_components].dot(np.diag(Sigma[0:n_components]).dot(V[:,0:n_components].T))
#     print(n_components, summ[n_components-1])
#     un = svd_un
    
    ### Wavelet denoising ###
#     wavelet_un = denoise_wavelet(un, channel_axis=None, convert2ycbcr=False, 
#                                  sigma=estimate_sigma(un, channel_axis=None, average_sigmas=True), 
#                                  method='BayesShrink', mode='soft', rescale_sigma=True)
#     un = wavelet_un
    
    ### Robust PCA ###
#     rpca_lam = 0.1
#     un, _ = RobustPCA(un, rpca_lam)
    
    ### FFT ###    
    def remove_f(uu, percent):
        if percent <= 0: return uu
        # PSD = (uu*np.conj(uu)) # also valid
        PSD = (uu*np.conj(uu))/np.prod(uu[0, :].shape)
        PSD = PSD.real
        mask = (PSD>=np.percentile(PSD, percent)).astype(np.float32)
        return uu*mask
    remove_percent = 50
    un = ifft(remove_f(fft(un), remove_percent)).real
    
    l_F, l_inf = np.linalg.norm(u-un, ord='fro'), np.linalg.norm(u-un, ord=np.inf)
    print(l_F, l_inf)

7.9994861519105465 5.248014032576636


#### Applying best-subset regression on the weak formulation

In [7]:
poly_deg = None; diff_order = 2
differentiation_method = ps.FiniteDifference; differentiation_kwargs = {}
kwargs = {'fit_intercept':False, 'copy_X':True, 'normalize_columns':False}

weak_pde_lib = ps.WeakPDELibrary(library_functions=[lambda x: x, lambda x: x * x], 
                                 function_names=[lambda x: x, lambda x: x + x], 
                                 derivative_order=diff_order, p=diff_order, 
                                 spatiotemporal_grid=XT, 
                                 include_bias=False, is_uniform=True, K=10000, # new random K points in every calls to the ps.WeakPDELibrary
                                 differentiation_method=differentiation_method, 
                                 differentiation_kwargs=differentiation_kwargs, 
                                 cache=True
                                )

X_pre, y_pre, fns = ps_features(un, t, weak_pde_lib, kwargs)

In [8]:
optimizer_name = "miosr" # "l0bnb", "miosr"
assert (optimizer_name == "l0bnb") or (optimizer_name == "miosr")
if optimizer_name == "l0bnb":
    optimizer = L0BNB(max_nonzeros=2, lam=1e-3, is_normal=True, normalize_columns=False)
elif optimizer_name == "miosr":
    optimizer = ps.MIOSR(target_sparsity=2, normalize_columns=True, regression_timeout=100)
    
n_ensemble_models = 50
model = ps.SINDy(feature_library=weak_pde_lib, optimizer=optimizer, 
                 differentiation_method=differentiation_method,
                 cache=True,
                 feature_names=['u'])
model.fit(np.expand_dims(un, -1), t=dt, ensemble=True, 
          library_ensemble=True, n_candidates_to_drop=1, n_models=n_ensemble_models)

# print found pde
print_pde(model.get_coef_list()[np.argmin(np.sum((np.squeeze(np.tensordot(X_pre, np.array(model.get_coef_list()).T, axes=([-1], [0])), axis=1)-y_pre)**2, axis=0))].reshape(-1,1), model.get_feature_names())

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-04
u_t = (0.095570 +0.000000i)u_11
    + (-1.007136 +0.000000i)uu_1
   


In [9]:
best_subsets = [tuple(np.where(abs(brute_force(X_pre, y_pre, support_size=i).flatten())>0)[0]) 
                for i in range(1, X_pre.shape[1]+1)]

#### Model selection by the UBIC (Algorithm 1)

In [10]:
tau = 3
verbose = True
scale = np.log(len(y_pre))
# scale = 1 <- generalized UBIC
per = 75

post_means, b_bics, b_uns = baye_uncertainties(best_subsets, (X_pre, y_pre), u_type='cv1', take_sqrt=True)
predictions = X_pre@post_means

b_bics = np.array(b_bics)
max_complexity = len(b_bics)
complexities = np.arange(max_complexity)+1
d_complexities = complexities[decreasing_values_indices(b_bics)]
d_bics = b_bics[decreasing_values_indices(b_bics)]
slopes = np.diff(b_bics)/(np.diff(complexities)*b_bics[:-1])
thres = np.percentile(np.abs(np.diff(d_bics)/(np.diff(d_complexities)*d_bics[:-1])), per)
# thres = 0.02
print("threshold:", thres)

lower_bounds = []
for k, efi in enumerate(best_subsets):
    assert len(efi) == np.count_nonzero(post_means[:, k:k+1])
    com = len(efi)
    lower_bound = 2*np.abs(log_like_value(predictions[:, k:k+1], y_pre))-np.log(len(y_pre))*com
    lower_bounds.append(lower_bound)

last_lam = np.log10(max(lower_bounds/(b_uns*scale)))
print("max_lam:", last_lam)
delta = last_lam/tau
now_lam = last_lam-delta
last_ubic = UBIC(b_bics, b_uns, len(y_pre), hyp=10**last_lam, scale=scale)
last_bc = np.argmin(last_ubic)
while now_lam >= 0:
    now_ubic = UBIC(b_bics, b_uns, len(y_pre), hyp=10**now_lam, scale=scale)
    now_bc = np.argmin(now_ubic)
    
    diff_com = now_bc-last_bc
    diff_bic = b_bics[now_bc]-b_bics[last_bc]
    imp = abs(diff_bic/(b_bics[last_bc]*diff_com))
    
    if verbose:
        print(min(last_bc, now_bc), '<--->', max(last_bc, now_bc), 
              np.nan_to_num(imp, nan=np.inf))
    
    if (diff_com > 0 and (diff_bic > 0 or imp < thres)) or \
        (diff_com < 0 and diff_bic > 0 and imp > thres):
        break
    
    last_lam = now_lam
    now_lam = round(last_lam-delta, 10)
    last_ubic = now_ubic
    last_bc = now_bc

best_bc = last_bc
if abs((b_bics[last_bc]-b_bics[last_bc-1])/b_bics[last_bc-1]) < thres:
    best_bc = best_bc - 1
    
last_lam = round(last_lam, 10)
last_lam, last_ubic, last_bc, best_bc

threshold: 0.061093751541539495
max_lam: 3.882633897336766
1 <---> 1 inf
1 <---> 1 inf
1 <---> 1 inf


(-0.0,
 array([-62632.1022474 , -70283.38479739, -70278.48472284, -70279.79876959,
        -70255.67170658, -70248.5392596 , -70220.97873975, -70042.86315962]),
 1,
 1)

In [11]:
complexity_axis = [len(bs) for bs in best_subsets]
with plt.style.context(['science']):
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    ax.set_zorder(ax2.get_zorder()+1)
    ax.patch.set_visible(False)
    
    l1, = ax.plot(complexity_axis, last_ubic, 'o-', c='black', markerfacecolor='none', label=f"$\lambda = {abs(last_lam)}$")
    ax.set_xticks(complexity_axis)
    ax.set_ylabel("$\\textrm{UBIC}(\\xi^{k}, 10^{\\lambda})$", fontsize=12)
    ax.set_xlabel("Support sizes ($s_{k}$)", fontsize=12)
    ax.annotate('$(u_{xx}, uu_{x})\checkmark$', fontsize=16, 
                 xy=(complexity_axis[best_bc], last_ubic[best_bc]), 
                 xytext=(complexity_axis[best_bc]+0.1, last_ubic[best_bc]+4000), 
                 arrowprops={'arrowstyle': '->'})
    
    l2, = ax2.plot(complexity_axis, b_uns, 'o--', c='blue', markerfacecolor='none', label="Uncertainty $\\textrm{U}^{k}$")
    s1 = ax2.scatter(complexity_axis[np.argmin(b_uns)], b_uns[np.argmin(b_uns)], c='blue')
    ax2.tick_params(axis='y', labelcolor='blue')

    ax.legend([l1, l2, s1], [f"UBIC with $\lambda = {abs(round(last_lam, 2))}$", "Uncertainty $\\textrm{U}^{k}$", "Min $\\textrm{U}^{k}$"], 
              labelcolor='linecolor', loc='upper center', fontsize=11.5)
    
    # plt.savefig("burgers_wavelet_ubic.pdf")
    plt.show()

#### Percent coefficient error

In [12]:
# print(n_components, summ[n_components-1])
print(l_F, l_inf)
evaluate_coefficients(np.linalg.lstsq(X_pre[:, best_subsets[best_bc]], 
                                      y_pre, rcond=None)[0].flatten())

7.9994861519105465 5.248014032576636


(0.7640200277319542, 0.08523892373680231)

In [13]:
# No denoising: 8.673577639876537 5.298318588581592 
# rksvd: 2.858361587460368 2.274811883362282 | (0.9107969701290103, 0.3772757947444888)
# svd n_comp=74: 8.2830942877307 4.990897588534944 | (0.4377375780166351, 0.18439574973366096)
# svd n_comp=86: 8.51040101515912 5.163254362785331 | (0.5747929503770288, 0.03141691852627804)
# svd n_comp=99 (99.5): 8.660746565852696 5.324954980348864 | (0.770910494195949, 0.1326679698575986)
# pca n_comp=3: 4.754262017591448 5.675915023400061 | - 3 selected
# pca n_comp=22: 5.539849551837206 3.837241763152806 | (1.4805624720730992, 1.36812501749769)
# pca 99.5: 8.411869854614082 5.054547005505603 | (0.5210652800684792, 0.08352530991912882)
# Robust PCA: 7.445189944826652 4.617636462159813 | (1.9538318222493618, 1.7621799177316084)
# wavelet: 2.5287134283765695 2.1318082910253775 | (2.4942281074157746, 1.5133039078568222)

In [14]:
# FFT_{10}: 5.226547898222989 5.066573466223977 | (1.6110419451166242, 1.0472379899339952)
# FFT_{25}: 6.753362788411607 5.163727170937074 | (1.201034558494282, 0.8659130290210453)
# FFT_{50} 7.9994861519105465 5.248014032576636 | (0.7640200277319542, 0.08523892373680231)

In [15]:
### SVD(90)
# 3 92.64923015787362
# 4.760708194694509 5.668854439173446
# Failed
### SVD(95)
# 11 95.11338531498691
# 3.9613742537410515 3.150348359135305
# (2.3853900084799093, 2.1872024582646086)
### SVD(99.5)
# 76 99.51829861634103
# 8.3282256755406 5.003217153027226
# (0.5228949159189394, 0.08261226057245386)
### SVD(99.99)
# 100 99.99112580092523
# 8.667298715625757 5.325563542724821
# (0.7780442175026381, 0.14434615548682972)